In [48]:
import pandas as pd

# Leer el archivo Parquet generado
df = pd.read_parquet("../data/interim/raw_concat.parquet")

# Mostrar las 5 primeras filas
df.head()


,Row Name,Clip Start,Clip End,Set_num,Set_num: Time,Set_P1,Set_P1: Time,Set_P2,Set_P2: Time,Servicio,Servicio: Time,Jugador,Jugador: Time,Pareja,Pareja: Time,Cara_pala,Cara_pala: Time,Pared,Pared: Time,Punto_lost,Punto_lost: Time,Winner,Winner: Time,Juego_lost,Juego_lost: Time,Punto_win,Punto_win: Time,Error,Error: Time,Juego_win,Juego_win: Time,Break_point,Break_point: Time,Fuerza error,Fuerza error: Time,Break_fav,Break_fav: Time,Punto_P1,Punto_P1: Time,Punto_P2,Punto_P2: Time,Set_point,Set_point: Time,Set_win,Set_win: Time,Set_lost,Set_lost: Time,Match point,Match point: Time,Zona saque,Zona saque: Time,Zona resto,Zona resto: Time,Inicio_gople,Inicio_gople: Time,Inicio_gople: X,Inicio_gople: Y,Juego_P1,Juego_P1: Time,Juego_P2,Juego_P2: Time,Break_con,Break_con: Time,Golpe_q,Golpe_q: Time,Fin_golpe,Fin_golpe: Time,Fin_golpe: X,Fin_golpe: Y,Asistencia,Asistencia: Time,__source_file
0,Servicio Federico Chingotto,77.011,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Federico Chingotto,78.011,Federico Chingotto,78.011,Chingotto - Galán,78.011,Derecha,79.218,Pared lat,79.218,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
1,Servicio Federico Chingotto,77.011,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Martín Di Nenno,79.218,Di Nenno - Ausburger,79.218,Derecha,81.005,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
2,Servicio Federico Chingotto,77.011,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Alejandro Galán,81.005,Chingotto - Galán,81.005,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
3,Servicio Federico Chingotto,77.011,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Chingotto - Galán,82.682,<NA>,<NA>,<NA>,<NA>,Di Nenno - Ausburger,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
4,Servicio Federico Chingotto,77.011,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Di Nenno - Ausburger,82.682,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...


In [49]:
import pandas as pd
df_clean = pd.read_parquet("../data/interim/final_clean.parquet")
df_clean.head()


,row_name,clip_start,clip_end,set_num,set_num:_time,set_p1,set_p1:_time,set_p2,set_p2:_time,servicio,servicio:_time,jugador,jugador:_time,pareja,pareja:_time,cara_pala,cara_pala:_time,pared,pared:_time,punto_lost,punto_lost:_time,winner,winner:_time,juego_lost,juego_lost:_time,punto_win,punto_win:_time,error,error:_time,juego_win,juego_win:_time,break_point,break_point:_time,fuerza_error,fuerza_error:_time,break_fav,break_fav:_time,punto_p1,punto_p1:_time,punto_p2,punto_p2:_time,set_point,set_point:_time,set_win,set_win:_time,set_lost,set_lost:_time,match_point,match_point:_time,zona_saque,zona_saque:_time,zona_resto,zona_resto:_time,inicio_gople,inicio_gople:_time,inicio_gople:_x,inicio_gople:_y,juego_p1,juego_p1:_time,juego_p2,juego_p2:_time,break_con,break_con:_time,golpe_q,golpe_q:_time,fin_golpe,fin_golpe:_time,fin_golpe:_x,fin_golpe:_y,asistencia,asistencia:_time,__source_file
0,1º Servicio,78.011,78.73,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Federico Chingotto,78.011,Federico Chingotto,78.011,Chingotto - Galán,78.011,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,78.338,1,78.338,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Servicio,78.011,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
1,1º Servicio,97.928,98.659,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Federico Chingotto,97.928,Federico Chingotto,97.928,Chingotto - Galán,97.928,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3,98.312,4,98.312,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Servicio,97.928,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
2,1º Servicio,118.469,119.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Federico Chingotto,118.469,Federico Chingotto,118.469,Chingotto - Galán,118.469,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,118.796,1,118.796,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Servicio,118.469,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
3,1º Servicio,135.256,135.987,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Federico Chingotto,135.256,Federico Chingotto,135.256,Chingotto - Galán,135.256,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3,135.64,4,135.64,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Servicio,135.256,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...
4,1º Servicio,192.954,193.684,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Martín Di Nenno,192.954,Martín Di Nenno,192.954,Di Nenno - Ausburger,192.954,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,193.464,1,193.464,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Servicio,192.954,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Milan_Semifinal_Chingalan_DinnenoAusburguer...


In [50]:
df_filtrado = df_clean.loc[
    df_clean["row_name"] == "Punto",    # condición de filas
    ["row_name", "punto_p1", "punto_p2"]  # columnas que quieres ver
]

In [51]:
df_filtrado

,row_name,punto_p1,punto_p2
854,Punto,0,0
855,Punto,15,0
856,Punto,30,0
857,Punto,40,0
858,Punto,0,0
...,...,...,...
1000,Punto,Adv,40
1001,Punto,0,0
1002,Punto,15,0
1003,Punto,30,0


In [47]:
df_filtrado.dtypes

row_name    string[python]
punto_p1    string[python]
punto_p2    string[python]
dtype: object